In [1]:
import json
from sqlalchemy import func, select, text
from sqlalchemy.orm import selectinload
from db.session import get_session
import pprint
import importlib
from models.dish import Dish

from models.menu import Menu
from models.submenu import Submenu
from schemas.menu import MenuCascadeDTO

statement = text(
            '''
        SELECT json_agg(main_menu) AS result
        FROM (
            SELECT
                m.id AS id,
                m.title AS title,
                m.description AS description,
                COUNT(DISTINCT(sub.id)) AS submenus_count,
                COALESCE(SUM(sub.dishes_count) :: bigint, 0) AS dishes_count,
                json_agg(sub) AS submenus
            FROM menu m
            LEFT JOIN (
                SELECT
                    s.id AS id,
                    s.title AS title,
                    s.description AS description,
                    s.menu_id,
                    COUNT(d.id) AS dishes_count,
                    json_agg(d) AS dishes
                FROM submenu s
                LEFT JOIN (
                    SELECT
                        d.id AS id,
                        d.title AS title,
                        d.description AS description,
                        d.price * (1 + d.discount) AS price,
                        d.submenu_id
                    FROM dish d
                ) d ON s.id = d.submenu_id
                GROUP BY s.id, s.title, s.description, s.menu_id
            ) sub ON m.id = sub.menu_id
            GROUP BY m.id, m.title, m.description
        ) main_menu;
        '''
        )

async def test_cascade():
    async with await get_session() as session:
        query = select(Menu).options(selectinload(Menu.submenus).options(selectinload(Submenu.dishes)))
        result = [MenuCascadeDTO.model_validate(menu, from_attributes=True).model_dump() for menu in (await session.execute(query)).scalars().all()]
        pprint.pprint(result)
    
await test_cascade()

[{'description': 'sada',
  'dishes_count': 1,
  'id': UUID('134397ee-4f56-4be0-b9c1-2c30bc01bd22'),
  'submenus': [{'description': 'asdasd',
                'dishes': [{'description': 'adsavvasdv',
                            'id': UUID('ab4397ee-4f56-4be0-b9c1-2c30bc01bd22'),
                            'price': Decimal('12.50'),
                            'submenu_id': UUID('da4397ee-4f56-4be0-b9c1-2c30bc01bd22'),
                            'title': 'bdasdba'}],
                'dishes_count': 1,
                'id': UUID('da4397ee-4f56-4be0-b9c1-2c30bc01bd22'),
                'menu_id': UUID('134397ee-4f56-4be0-b9c1-2c30bc01bd22'),
                'title': 'adva'}],
  'submenus_count': 1,
  'title': 'asd'},
 {'description': 'da',
  'dishes_count': 0,
  'id': UUID('134397ee-4f56-4be0-b9c1-2c30bc01bd21'),
  'submenus': [],
  'submenus_count': 0,
  'title': '2das'}]


In [5]:
from background.tasks import update_menus
import nest_asyncio

nest_asyncio.apply()

update_menus()

2023-08-14 01:38:21,367 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-14 01:38:21,369 INFO sqlalchemy.engine.Engine UPDATE menu SET title=$1::VARCHAR, description=$2::VARCHAR WHERE menu.id = $3::UUID RETURNING menu.id, menu.title, menu.description
2023-08-14 01:38:21,371 INFO sqlalchemy.engine.Engine [cached since 187.1s ago] ('Меню', 'Основное меню', UUID('2fe28da6-3d07-4797-b5c3-0c71ba018b5b'))
2023-08-14 01:38:21,779 INFO sqlalchemy.engine.Engine SELECT count(distinct(submenu.id)) AS count_1, count(dish.id) AS count_2, menu.id 
FROM menu LEFT OUTER JOIN submenu ON menu.id = submenu.menu_id LEFT OUTER JOIN dish ON submenu.id = dish.submenu_id 
WHERE menu.id = $1::UUID GROUP BY menu.id
2023-08-14 01:38:21,781 INFO sqlalchemy.engine.Engine [cached since 187.2s ago] (UUID('2fe28da6-3d07-4797-b5c3-0c71ba018b5b'),)
2023-08-14 01:38:21,981 INFO sqlalchemy.engine.Engine COMMIT
2023-08-14 01:38:22,086 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-14 01:38:22,088 INFO sqlalc